In [1]:
!pip install tensorflow streamlit pyngrok numpy pillow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.4 MB/s eta 0:00:00


In [3]:
%%writefile app.py

import streamlit as st
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import os

# Load the trained model
MODEL_PATH = "/content/densenet_model.h5"
if not os.path.exists(MODEL_PATH):
    st.error("⚠️ Model file not found! Please upload 'densenet_model.h5'.")
    st.stop()

model = load_model(MODEL_PATH)

# Define image properties
IMG_SIZE = (224, 224)

# Class Labels
CLASS_LABELS = [
    "8_celled_grade_a", "8_celled_grade_b", "8_celled_grade_c",
    "morula_grade_a", "morula_grade_b", "morula_grade_c",
    "blastocyst_grade_a", "blastocyst_grade_b", "blastocyst_grade_c"
]

# Streamlit UI
st.title("🔬 Embryo Classification App")
st.write("Upload an image, and the model will classify the embryo stage and grade. If the image is not an embryo, it will return **'Not a Valid Image'.**")

# Upload image
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "png", "jpeg"])

def preprocess_image(img_path):
    """ Preprocess the image for model prediction. """
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def predict_image(img_path):
    """ Predict the image and return the class or 'Not a Valid Image'. """
    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)

    confidence_threshold = 0.6  # Adjust threshold for better OOD detection
    max_prob = np.max(prediction)

    if max_prob < confidence_threshold:
        return "Not a Valid Image"

    predicted_class = np.argmax(prediction)
    return CLASS_LABELS[predicted_class]

if uploaded_file:
    st.image(uploaded_file, caption="Uploaded Image", use_column_width=True)

    # Save file temporarily
    temp_path = "temp.jpg"
    with open(temp_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    # Prediction
    st.write("🧐 **Processing Image...**")
    prediction_result = predict_image(temp_path)

    # Display Result
    st.subheader(f"✅ Predicted Class: **{prediction_result}**")


Overwriting app.py


In [4]:
!nohup streamlit run app.py --server.port 8501 --server.headless true > streamlit.log 2>&1 &


In [5]:
from pyngrok import ngrok

NGROK_AUTH_TOKEN = "2tHYZPUNqzf827xMIIKnbqSeOEW_4UxTAnyJAjztKmoWjC4Uo"  # Replace with your ngrok token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(8501)

print(f"🚀 Streamlit app is live at: {public_url}")


🚀 Streamlit app is live at: NgrokTunnel: "https://ddb5-34-125-140-214.ngrok-free.app" -> "http://localhost:8501"
